In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.io import fits
from astropy.utils.data import download_file

In [2]:
from astroquery.mast import Observations
import tarfile
from astropy.table import Table

In [5]:
koi_table = Table.read('data/cumulative_2019.07.05_01.52.59.votable')

In [6]:
# 0.5 mmag -> flux [ppm]
half_mmag = (10**((0.5e-3) / 2.5) - 1) * 1e6
print(half_mmag)

460.6230728403382


In [43]:
shortlist = koi_table[(koi_table['koi_steff'] < 6000) & (koi_table['koi_depth'] > half_mmag) & 
                      (koi_table['koi_kepmag'] < 14.2) & (koi_table['koi_disposition'] != b'FALSE POSITIVE')]

/Users/bmmorris/anaconda/lib/python3.5/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in less
  return getattr(self.data, op)(other)
/Users/bmmorris/anaconda/lib/python3.5/site-packages/astropy/table/column.py:965: RuntimeWarning: invalid value encountered in greater
  return getattr(self.data, op)(other)


In [44]:
kepids = [i.decode() for i in shortlist['kepid'].data.data]

In [45]:
# from glob import glob
# import shutil
# for path in glob('data/kepler_lcs/?????????'): 
#     kepid = "{0}".format(int(path.split(os.sep)[-1]))
#     if kepid not in kepids: #os.path.exists("data/kepler_lcs/{0:09d}".format(int(kepid))): 
#         print(kepid)
#         shutil.rmtree(path)

In [54]:
from bs4 import BeautifulSoup

In [72]:
kepids = ['001026895']
for kepid in kepids:
    if not os.path.exists('data/kepler_lcs/{0:09d}'.format(int(kepid))):
        print(kepid)

        obs = Observations.query_criteria(filters=["Kepler"], instrument_name='Kepler', 
                                          objectname="KIC {0}".format(kepid), radius=0.05, 
                                          dataproduct_type='timeseries', t_exptime=[50, 70])

        
        if len(obs['dataURL'].data.data) > 0:
            path = download_file(obs['dataURL'].data.data[0])

            tar = tarfile.open(path)
            tar.extractall('data/kepler_lcs/.')
            tar.close()
            
#         else: 
#             print(kepid)
#             prefix_url = 'https://archive.stsci.edu/missions/kepler/lightcurves/{0}/{1:09d}'.format("{0:09d}".format(int(kepid))[:4], int(kepid))
#             path = download_file(prefix_url)
#             bs = BeautifulSoup(open(path, 'r'))
#             for a in bs.find_all('a'): 
#                 if a.get('href').endswith('.tar'): 
#                     suffix_url = a.get('href')
                    
#             url = prefix_url + '/' + suffix_url
#             print(url)
#             path = download_file(url)

#             tar = tarfile.open(path)
#             tar.extractall('data/kepler_lcs/.')
#             tar.close()

001026895
